In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# 폴더 경로 지정
folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017'

In [2]:
benign_folder_path = r'C:\Users\pc\Desktop\CNN\CIC-AndMal2017\Benign'

# 'Benign' 클래스의 파일 경로 리스트 받아오기
benign_file_paths = glob.glob(benign_folder_path + '/*.csv')

# 'Benign' 클래스의 데이터프레임 초기화
benign_data = pd.DataFrame()

# 'Benign' 클래스의 모든 파일 로드
for file_path in benign_file_paths:
    data = pd.read_csv(file_path)
    benign_data = pd.concat([benign_data, data], ignore_index=True)

# 'Benign' 클래스에서 160,000개 무작위 샘플 추출
num_samples_benign = 160000
labels = []

if len(benign_data) > num_samples_benign:
    train_benign_subset = benign_data.sample(n=num_samples_benign, random_state=42)
else:
    train_benign_subset = benign_data.copy()

# Benign 데이터의 라벨을 labels 리스트에 추가
labels.extend(train_benign_subset[' Label'].tolist())

print("train_benign_subset의 샘플 개수:", len(train_benign_subset))

train_benign_subset의 샘플 개수: 160000


In [3]:
ransomware_folder_list = glob.glob(folder_path + '\\Ransomware\\*')
all_ransomware_data = pd.DataFrame()

for ransomware_folder_path in ransomware_folder_list:
    ransomware_file_paths = glob.glob(ransomware_folder_path + '/*.csv')
    
    # 각 랜섬웨어 클래스의 모든 파일 로드
    for file_path in ransomware_file_paths:
        data = pd.read_csv(file_path)
        all_ransomware_data = pd.concat([all_ransomware_data, data], ignore_index=True)

# 전체 랜섬웨어 데이터에서 160,000개 샘플 무작위로 추출
if len(all_ransomware_data) > 160000:
    train_ransomware_subset = all_ransomware_data.sample(n=160000, random_state=42, replace=False)
else:
    train_ransomware_subset = all_ransomware_data.copy()

# 랜섬웨어 데이터의 라벨을 labels 리스트에 추가
labels.extend(train_ransomware_subset[' Label'].tolist())

print("train_ransomware_subset의 샘플 개수:", len(train_ransomware_subset))

train_ransomware_subset의 샘플 개수: 160000


In [4]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
train_dataset = pd.concat([train_benign_subset, train_ransomware_subset], ignore_index=True)

print("최종 train 데이터세트의 샘플 개수:", len(train_dataset))

최종 train 데이터세트의 샘플 개수: 320000


In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# 상관분석으로 선택된 특성들만 사용
selected_features = [
    ' Active Max',
    ' Avg Bwd Segment Size',
    ' Bwd Header Length',
    ' Bwd Packet Length Mean',
    ' Fwd Header Length',
    ' Packet Length Variance',
    ' Subflow Bwd Packets',
    ' Subflow Fwd Bytes',
    ' Total Backward Packets',
    ' Total Fwd Packets',
    ' act_data_pkt_fwd',
    ' min_seg_size_forward',
    'Active Mean',
    'Subflow Fwd Packets',
    'Total Length of Fwd Packets'
]

# 선택된 특성만 포함하는 데이터프레임 생성
X_selected = train_dataset[selected_features]

# 데이터 스케일링
scaler = MinMaxScaler()
scaler.fit(train_dataset[selected_features])  # 스케일러 학습
X_scaled = scaler.transform(train_dataset[selected_features])  # 훈련 데이터에 스케일러 적용

# 정규화된 특성들을 데이터프레임으로 변환
X_normalized = pd.DataFrame(X_scaled, columns=selected_features)

# 타깃 변수 데이터로 변환
y = np.array(labels)

# 결과 출력
print("정규화된 선택된 특성들의 데이터프레임:")
print(X_normalized.head())

정규화된 선택된 특성들의 데이터프레임:
    Active Max   Avg Bwd Segment Size   Bwd Header Length  \
0     0.000000               0.070548            0.999152   
1     0.000000               0.000000            0.999152   
2     0.001415               0.025616            0.999152   
3     0.000000               0.089212            0.999152   
4     0.000000               0.000000            0.999152   

    Bwd Packet Length Mean   Fwd Header Length   Packet Length Variance  \
0                 0.070548            0.993129                 0.000198   
1                 0.000000            0.993129                 0.000000   
2                 0.025616            0.993129                 0.003535   
3                 0.089212            0.993129                 0.019651   
4                 0.000000            0.993129                 0.000000   

    Subflow Bwd Packets   Subflow Fwd Bytes   Total Backward Packets  \
0              0.000067            0.000004                 0.000067   
1              0

In [6]:
# 각 행의 마지막에 0 값을 추가합니다.
X_padded = np.pad(X_normalized.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 선택된 특성을 4x4 크기의 2차원 행렬로 변환
num_rows = 4
num_columns = 4
X_final_reshaped = X_padded.reshape(-1, num_rows, num_columns)

# 결과 출력
print("4x4 크기의 2차원 행렬:")
for i in range(num_rows):
    print(X_final_reshaped[0][i])  # 첫 번째 샘플에 해당하는 2x3 행렬 출력

4x4 크기의 2차원 행렬:
[0.         0.07054795 0.99915187 0.07054795]
[9.93128604e-01 1.98311196e-04 6.70690812e-05 4.00783933e-06]
[6.70690812e-05 0.00000000e+00 0.00000000e+00 8.91079304e-01]
[0.00000000e+00 0.00000000e+00 4.00783933e-06 0.00000000e+00]


In [7]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Train_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_final_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_final_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 160000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [8]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Train_Image\\'
X_data = []
y_labels = []

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_data.append(image)
        y_labels.append(label_int)

X_data = np.array(X_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장
y_labels = np.array(y_labels)

In [9]:
# train_benign_subset에 포함되지 않은 데이터 선택
validation_data_candidate = benign_data.drop(train_benign_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_validation = 20000
if len(validation_data_candidate) > num_samples_validation:
    validation_benign_subset = validation_data_candidate.sample(n=num_samples_validation, random_state=42)
else:
    validation_benign_subset = validation_data_candidate.copy()

print("validation_benign_subset의 샘플 개수:", len(validation_benign_subset))

validation_benign_subset의 샘플 개수: 20000


In [10]:
# train_ransomware_subset에 포함되지 않은 랜섬웨어 데이터 선택
validation_ransomware_candidate = all_ransomware_data.drop(train_ransomware_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_validation_ransomware = 20000
if len(validation_ransomware_candidate) > num_samples_validation_ransomware:
    validation_ransomware_subset = validation_ransomware_candidate.sample(n=num_samples_validation_ransomware, random_state=42)
else:
    validation_ransomware_subset = validation_ransomware_candidate.copy()

print("validation_ransomware_subset의 샘플 개수:", len(validation_ransomware_subset))

validation_ransomware_subset의 샘플 개수: 20000


In [11]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
validation_dataset = pd.concat([validation_benign_subset, validation_ransomware_subset], ignore_index=True)

print("최종 validation 데이터세트의 샘플 개수:", len(validation_dataset))

최종 validation 데이터세트의 샘플 개수: 40000


In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# 상관분석으로 선택된 특성들만 사용
selected_features = [
    ' Active Max',
    ' Avg Bwd Segment Size',
    ' Bwd Header Length',
    ' Bwd Packet Length Mean',
    ' Fwd Header Length',
    ' Packet Length Variance',
    ' Subflow Bwd Packets',
    ' Subflow Fwd Bytes',
    ' Total Backward Packets',
    ' Total Fwd Packets',
    ' act_data_pkt_fwd',
    ' min_seg_size_forward',
    'Active Mean',
    'Subflow Fwd Packets',
    'Total Length of Fwd Packets'
]

# 선택된 특성만 포함하는 데이터프레임 생성
val_X_selected = validation_dataset[selected_features]

# 앞서 훈련 데이터에 대해 학습한 MinMaxScaler 인스턴스를 사용하여 검증 데이터를 변환
val_X_scaled = scaler.transform(val_X_selected)

# 정규화된 특성들을 데이터프레임으로 변환
val_X_normalized = pd.DataFrame(val_X_scaled, columns=selected_features)

# 타깃 변수 데이터로 변환
y = np.array(labels)

# 결과 출력
print("정규화된 선택된 특성들의 데이터프레임:")
print(val_X_normalized.head())

정규화된 선택된 특성들의 데이터프레임:
    Active Max   Avg Bwd Segment Size   Bwd Header Length  \
0          0.0               0.339937            0.999152   
1          0.0               0.000000            0.999152   
2          0.0               0.000000            0.999152   
3          0.0               0.168493            0.999152   
4          0.0               0.000000            0.999152   

    Bwd Packet Length Mean   Fwd Header Length   Packet Length Variance  \
0                 0.339937            0.993129                 0.036217   
1                 0.000000            0.993129                 0.000000   
2                 0.000000            0.993129                 0.000000   
3                 0.168493            0.993129                 0.001898   
4                 0.000000            0.993129                 0.000000   

    Subflow Bwd Packets   Subflow Fwd Bytes   Total Backward Packets  \
0              0.000872            0.000391                 0.000872   
1              0

In [13]:
# 각 행의 마지막에 0 값을 추가합니다.
X_val_padded = np.pad(val_X_normalized.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 선택된 특성을 4x4 크기의 2차원 행렬로 변환
num_rows = 4
num_columns = 4
X_val_reshaped = X_val_padded.reshape(-1, num_rows, num_columns)

# 결과 출력
print("4x4 크기의 2차원 행렬:")
for i in range(num_rows):
    print(X_val_reshaped[0][i]) 

4x4 크기의 2차원 행렬:
[0.         0.33993678 0.99915187 0.33993678]
[9.93128606e-01 3.62174087e-02 8.71898055e-04 3.90918483e-04]
[8.71898055e-04 1.48500149e-03 8.91000891e-04 8.91079311e-01]
[0.         0.001485   0.00039092 0.        ]


In [14]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Validation_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_val_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_val_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 20000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [15]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Validation_Image\\'  # Modified path to "Validation_Image"

X_val_data = []  # Changed variable name to X_val_data for clarity
y_val_labels = []  # Changed variable name to y_val_labels for clarity

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_val_data.append(image)  # Changed variable name
        y_val_labels.append(label_int)  # Changed variable name

X_val_data = np.array(X_val_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장, changed variable name
y_val_labels = np.array(y_val_labels)  # Changed variable name

In [16]:
# train_benign_subset 및 validation_benign_subset에 포함되지 않은 데이터 선택
test_data_candidate = benign_data.drop(train_benign_subset.index).drop(validation_benign_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_test = 20000
if len(test_data_candidate) > num_samples_test:
    test_benign_subset = test_data_candidate.sample(n=num_samples_test, random_state=42)
else:
    test_benign_subset = test_data_candidate.copy()

print("test_benign_subset의 샘플 개수:", len(test_benign_subset))

test_benign_subset의 샘플 개수: 20000


In [17]:
# validation_ransomware_subset에 포함되지 않은 랜섬웨어 데이터 선택
test_ransomware_candidate = validation_ransomware_candidate.drop(validation_ransomware_subset.index)

# 선택된 데이터 중에서 20,000개를 무작위로 선택
num_samples_test_ransomware = 20000
if len(test_ransomware_candidate) > num_samples_test_ransomware:
    test_ransomware_subset = test_ransomware_candidate.sample(n=num_samples_test_ransomware, random_state=42)
else:
    test_ransomware_subset = test_ransomware_candidate.copy()

print("test_ransomware_subset의 샘플 개수:", len(test_ransomware_subset))

test_ransomware_subset의 샘플 개수: 20000


In [18]:
# 'Benign' 클래스와 랜섬웨어 클래스의 데이터프레임들을 합치기
test_dataset = pd.concat([test_benign_subset, test_ransomware_subset], ignore_index=True)

print("최종 test 데이터세트의 샘플 개수:", len(test_dataset))

최종 test 데이터세트의 샘플 개수: 40000


In [19]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# 상관분석으로 선택된 특성들만 사용
selected_features = [
    ' Active Max',
    ' Avg Bwd Segment Size',
    ' Bwd Header Length',
    ' Bwd Packet Length Mean',
    ' Fwd Header Length',
    ' Packet Length Variance',
    ' Subflow Bwd Packets',
    ' Subflow Fwd Bytes',
    ' Total Backward Packets',
    ' Total Fwd Packets',
    ' act_data_pkt_fwd',
    ' min_seg_size_forward',
    'Active Mean',
    'Subflow Fwd Packets',
    'Total Length of Fwd Packets'
]

# 선택된 특성만 포함하는 데이터프레임 생성
test_X_selected = test_dataset[selected_features]

# 앞서 훈련 데이터에 대해 학습한 MinMaxScaler 인스턴스를 사용하여 테스트 데이터를 변환
test_X_scaled = scaler.transform(test_X_selected)

# 정규화된 특성들을 데이터프레임으로 변환
test_X_normalized = pd.DataFrame(test_X_scaled, columns=selected_features)

# 타깃 변수 데이터로 변환
y = np.array(labels)

# 결과 출력
print("정규화된 선택된 특성들의 데이터프레임:")
print(test_X_normalized.head())

정규화된 선택된 특성들의 데이터프레임:
    Active Max   Avg Bwd Segment Size   Bwd Header Length  \
0          0.0               0.341324            0.999152   
1          0.0               0.434075            0.999152   
2          0.0               0.165753            0.999152   
3          0.0               0.394178            0.999152   
4          0.0               0.000000            0.999152   

    Bwd Packet Length Mean   Fwd Header Length   Packet Length Variance  \
0                 0.341324            0.993129                 0.038383   
1                 0.434075            0.993129                 0.042979   
2                 0.165753            0.993129                 0.002015   
3                 0.394178            0.993129                 0.057352   
4                 0.000000            0.993129                 0.000000   

    Subflow Bwd Packets   Subflow Fwd Bytes   Total Backward Packets  \
0              0.000805            0.000247                 0.000805   
1              0

In [20]:
test_X_padded = np.pad(test_X_normalized.values, ((0, 0), (0, 1)), mode='constant', constant_values=0)

# 선택된 특성을 4x4 크기의 2차원 행렬로 변환
num_rows = 4
num_columns = 4
X_test_reshaped = test_X_normalized.values.reshape(-1, num_rows, num_columns)

# 결과 출력
print("4x4 크기의 2차원 행렬:")
for i in range(num_rows):
    print(X_test_reshaped[0][i])  # 첫 번째 샘플에 해당하는 2x3 행렬 출력

4x4 크기의 2차원 행렬:
[0.         0.3413242  0.99915187 0.3413242 ]
[9.93128606e-01 3.83833126e-02 8.04828974e-04 2.47355622e-04]
[8.04828974e-04 1.63350163e-03 5.94000594e-04 8.91079311e-01]
[0.         0.0016335  0.00024736 0.        ]


In [21]:
from PIL import Image
import os

# 이미지를 저장할 폴더를 생성합니다.
save_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Test_Image\\'
os.makedirs(os.path.join(save_folder, "Benign"), exist_ok=True)
os.makedirs(os.path.join(save_folder, "Ransomware"), exist_ok=True)

# 이미지 확대 비율 설정
resize_factor = 10  # 10배 확대

# X_final_reshaped를 이미지로 변환하여 저장합니다.
num_samples = X_test_reshaped.shape[0]
for i in range(num_samples):
    # 1채널 16비트 PNG 이미지 생성
    image_data = X_test_reshaped[i]
    image_data = (image_data * 65535).astype('uint16')  # 16비트로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환

    # 이미지 데이터 확대
    image_data = np.repeat(image_data, resize_factor, axis=0)  # 행 확대
    image_data = np.repeat(image_data, resize_factor, axis=1)  # 열 확대
    
    # 확대된 데이터를 다시 이미지로 변환
    image = Image.fromarray(image_data, 'I;16')  # 16비트 단일 채널 이미지로 변환
    
    # 이미지의 라벨에 따라서 저장 위치 및 파일명 설정
    if i < 20000:
        label = "Benign"
    else:
        label = "Ransomware"

    image_filename = f"image_{i}.png"
    image_filepath = os.path.join(save_folder, label, image_filename)

    # 해당 경로에 이미지가 이미 존재하는 경우 건너뛴다.
    if os.path.exists(image_filepath):
        continue

    image.save(image_filepath)

print("이미지 변환 및 저장이 완료되었습니다.")

이미지 변환 및 저장이 완료되었습니다.


In [22]:
import os
import cv2
import numpy as np

# 라벨 목록 및 해당 라벨에 대한 정수 값 지정
label_mapping = {
    "Benign": 0,
    "Ransomware": 1
}

# 이미지 데이터를 저장한 폴더 경로
image_folder = r'C:\Users\pc\Desktop\CNN\Correlation\0.25_threshold\Test_Image\\'  # Modified path to "Test_Image"

X_test_data = []  # Changed variable name to X_test_data for clarity
y_test_labels = []  # Changed variable name to y_test_labels for clarity

# 모든 라벨마다 폴더를 순회합니다.
for label_str, label_int in label_mapping.items():
    folder_path = os.path.join(image_folder, label_str)  # 해당 라벨의 폴더 경로
    
    # 해당 폴더 내의 모든 이미지 파일을 순차적으로 처리
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        
        # 16-bit 이미지 로딩
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:  # 이미지 로딩 실패
            print(f"Failed to load image at path: {image_path}")
            continue
        
        X_test_data.append(image)  # Changed variable name
        y_test_labels.append(label_int)  # Changed variable name

X_test_data = np.array(X_test_data, dtype=np.uint16)  # uint16 형태로 이미지 데이터 저장, changed variable name
y_test_labels = np.array(y_test_labels)  # Changed variable name